# Exercise 5.1 Least Squares Global Localization

## Theoretical background

In this lesson, we will use the Least Squares algorithm to locate our robot given a set of known landmarks.
Least Squares is akin to solve a system of equations of the form:

$$z_{m \times 1} = H_{m \times n} \cdot x_{n \times 1}$$

Where $n$ is the length of the pose, and $m$ the number of observations.

This simple concept, nevertheless, has to be modified in order to be used to solve our problem:

1. **Pseudo-inverse**. Generally, to solve an equation system, we only need as many equations (or in our case observations, $z$) as variables (in our case components of the state/pose, The typical approach is to drop some of the additional equations, but we have to assume that $z$ has been affected by some noise, we may use the additional information to correct the noise.

  The solution: we make use of the *pseudo-inverse* of a matrix. This, in contrast to the normal inverse operation, can be used in non-square matrices. This operation will result in the closest possible $\hat{x}$, i.e. the one where the the error is minimal.
  
  $$\hat{x} = \underbrace{(H^T H)^{-1} H^T}_{\textit{pseudo-inverse }(H^+)} z$$

2. **Weighted measurements** (optional). Not all observations are created equally, in some cases we can introduce how important each measurement is. For example, in this assignment, our sensor's accuracy drops the further the observed landmark is. Given a *covariance* matrix $Q$ describing the error in the measurements, the equations above changes:

  $$
    \begin{aligned}
      &\hat{x} \leftarrow (H^T Q^{-1} H)^{-1} H^T Q^{-1} z &\text{(1. Best estimation)}\\ 
      &\Sigma_{\hat{x}} \leftarrow (H^T Q^{-1} H)^{-1} &\text{(2. Uncertainty of the estimation)}\\
    \end{aligned}
  $$
  
3. **Non-linear Least Squares**. Until now we have assumed $\hat{x}$ can be solved as a simple system of equations, i.e. $H$ is a matrix. Nevertheless, our observation model is non linear: $z = h(x)$. Therefore, the only solution available is iterative: the *Gauss-Newton algorithm*.

  $$
  \begin{aligned}
      &\hat{x} \leftarrow (\dots)  &\text{(1. Initial guess)} \\
      &\delta \leftarrow (J_e^T Q^{-1} J_e)^{-1} J_e^T  Q^{-1} e  &\text{(2. Evaluate delta/increment)} \\
      &\hat{x} \leftarrow \hat{x} - \delta &\text{(3. Update estimation)} \\
      &\textit{if }\delta > \textit{tolerance} \rightarrow \textit{goto (1.)} \\
      &\textit{else } \rightarrow \textit{return }\hat{x} &\text{(4. Exit condition)}\\
  \end{aligned}
  $$
  
  Where:
  
  - $Q$ is the measurement covariance (*weighted measurement*)
  - $J_e$ is the Jacobian of the observation model at $\hat{x}$, also known as $\nabla h_{\hat{x}}$
  - $e$ is the error of $z$ against $h(\hat{x})$ (computed using the map information).
  
**Assignment**  

The attached code partially implements the global localization of a robot equipped with a range
sensor by means of Least Squares. This is done by the following steps:

1. The map is built. In this case, the map consists of a number of landmarks (`nLandmarks`).
2. The program asks the user to set the true position of the robot (`xTrue`) by clicking with the mouse in the map.
3. A new pose is generated from it, `xOdom`, which represents the pose that the robot thinks it is in. This simulates a motion command from an arbitrary pose that ends up with the robot in `xTrue`, but it thinks that it is in `xOdom`.
4. Then the robot takes a range measurement to each landmark in the map.
5. Finally, the robot employs a Least Squares definition of the problem and Gauss-Newton to iteratively optimize such a guess, obtaining a new (and hopefully better) estimation of its pose `xEst`.

**Example**

The figure below shows an example of execution of this code (once completed).

<figure style="text-align:center">
  <img src="images/fig5-1-1.png" width="400" alt="">
  <!--figcaption>Fig. 1: </figcaption-->
</figure>

In [ ]:
#%matplotlib notebook
#%matplotlib inline

# IMPORTS

import math

import numpy as np
from numpy import linalg
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
import scipy
from scipy import stats

from utils.PlotEllipse import PlotEllipse
from utils.DrawRobot import DrawRobot
from utils.tcomp import tcomp
from utils.tinv import tinv, jac_tinv1 as jac_tinv
from utils.Jacobians import J1, J2

To simplify a bit the code we'll provide you the following classes:

In [ ]:
class Robot(object):
    """ Simulate a robot base and positioning.
    
        Attrs:
            pose: Position given by odometry (in this case true_pose affected by noise)
            true_pose: True position, selected by the mouse in this demo
            cov: Covariance for the odometry sensor. Used to add noise to pose
            var_d: Covariance (noise) of each range measurement
    """
    def __init__(self,
                 pose: np.ndarray,
                 cov: np.ndarray,
                 desv_d: int = 0):
        # Pose related
        
        self.true_pose = pose
        self.pose = pose + np.sqrt(cov)@scipy.randn(3, 1)
        self.cov = cov
        
        # Sensor related
        self.var_d = desv_d**2
    
    def plot(self, fig, ax, **kwargs):
        DrawRobot(fig, ax, self.pose, color='red', label="Pose estimation (Odometry)", **kwargs)
        DrawRobot(fig, ax, self.true_pose, color="blue", label="Real pose", **kwargs)

Implement the following function to simulate how our robot observes the world. In this practice, our landmarks act as beacons. We can sense how far away they are without knowing the angles.

In [ ]:
def distance(pose: np.ndarray, m: np.ndarray, cov_d: int = 0) -> np.ndarray:
    """ Get observations for every landmark in the map.

    In this case our observations are range only.
    If cov_d > 0 then add gaussian noise with var_d covariance

    Args:
        pose: pose (true or noisy) of the robot taking observation
        m: Map containing all landmarks
        cov_d: Covariance of the sensor

    Returns
        z: numpy array containing distances to all obs. It has shape (nLandmars, 1). 
    """
    # TODO: Complete function
    raise NotImplementedError

Finally, we get to implement the Least Squares algorithm for localization. We ask you to complete the gaps in the following function.

In [ ]:
def LeastSquaresLocalization(R1: Robot,
                             w_map: np.ndarray,
                             z: np.ndarray,
                             nIterations=10,
                             tolerance=0.001,
                             delay=0.5) -> np.ndarray:
    """ Pose estimation using Gauss-Newton for least squares optimization
    
        Args:
            R1: Robot which pose we must estimate
            w_map: Map of the environment
            z: Observation received from sensor
            
            nIterations: sets the maximum number of iterations (default 10)
            tolerance: Minimum error needed for stopping the loop (convergence) (default 0.001)
            
            delay: Wait time used to visualize the different iterations (default 0.5)
            
        Returns:
            xEst: Estimated pose
    
    """
    
    iteration = 0
    
    # Initialization of useful variables
    incr = np.ones((2, 1)) # Delta
    jH = np.zeros((w_map.shape[1], 2)) # Jacobian of the observation function of all the landmarks
    xEst = R1.pose #Initial estimation is the odometry position (usually noisy)
    
    # Let's go!
    while linalg.norm(incr) > tolerance and iteration < nIterations:
        #if plotting:
        plt.plot(xEst[0], xEst[1], '+r', markersize=1+math.floor((iteration*15)/nIterations))
        # Compute the predicted observation (from xEst) and their respective Jacobians

        # 1) TODO: Compute distance to each landmark from xEst (estimated observations w/o noise)
        # 
        zEst = None

        # error = difference between real observations and predticed ones.
        e = None
        residual = np.sqrt(e.T@e) #residual error = sqrt(x^2+y^2)

        # 2) TODO: Compute Jacobians with respect (x,y) (slide 13)
        # The jH is evaluated at our current guest (xEst) -> z_p
        
        jH = None

        # The observation variances R grow with the root of the distance
        R = np.diag(R1.var_d*np.sqrt(z))

        # 3) TODO: Solve the equation --> compute incr
        incr = None
        
        plt.plot([xEst[0, 0], xEst[0, 0]+incr[0]], [xEst[1, 0], xEst[1, 0]+incr[1]], 'r')
        xEst[0:2, 0] += incr
        iteration += 1
        plt.pause(delay)

    plt.plot(xEst[0, 0], xEst[1, 0], '*g', markersize=14, label="Final estimation") #The last estimation is plot in green
    
    return xEst

Test your function it with the main bellow. Then answer the questions:
 
- Which is the minimum number of landmarks needed for localizing the robot? Why?
- Play with different “qualities” of the range sensor. Could you find a value for its variance so the LS method fails?
- Play also with different values for the odometry uncertainty.

In [ ]:
def main(nLandmarks=7, env_size=140):
    # MATPLOTLIB
    fig, ax = plt.subplots()
    plt.xlim([-70, 70])
    plt.ylim([-70, 70])
    plt.grid()
    plt.ion()
    plt.tight_layout()

    fig.canvas.draw()
    
    # VARIABLES
    num_landmarks=7
    env_size=140
    
    w_map = env_size*scipy.rand(2, num_landmarks) - env_size/2 
    ax.plot(w_map[0, :], w_map[1, :], 'o', color='magenta', label="Landmarks")
    
    xStart = np.vstack(plt.ginput(1)).T
    R1 = Robot(pose=np.vstack([xStart, 0]), cov=np.diag([15, 20, np.pi*180])**2, desv_d=1.5)
    R1.plot(fig, ax)

    # MAIN
    z = distance(R1.true_pose, w_map, cov_d=R1.var_d)
    LeastSquaresLocalization(R1, w_map, z)
    
    # PLOTTING
    plt.legend()
    fig.canvas.draw()
    
main()